In [1]:
import os
import sys 

import pdblp
import pandas as pd

In [2]:
blp_path = r"C:\blpapi_cpp_3.16.6.1"
os.environ["PATH"] = f'{os.environ["PATH"]}{blp_path}'

In [3]:
data = pd.read_csv('dataset.csv', index_col=0, sep=";")

date_cols = ['ISSUE DATE', 'MATURITY']
for col in date_cols:
    data[col] = pd.to_datetime(data[col])
    
data=data.loc[data["DEFAULT DATE"].isna(), :].reset_index(drop=True)
data

ISIN               ISSUER NAME  Ticker ISSUE DATE   MATURITY  \
0     US629568AX43     NABORS INDUSTRIES INC  NBR US 2012-02-13 2021-09-15   
1     US539830AY52      LOCKHEED MARTIN CORP  LMT US 2011-09-09 2021-09-15   
2     US713448BW74               PEPSICO INC  PEP US 2011-08-29 2021-08-25   
3     US745310AD42          PUGET ENERGY INC  PSD US 2011-10-08 2021-01-09   
4     US25468PCN42  TWDC ENTERPRISES 18 CORP  DIS US 2011-08-22 2021-08-16   
...            ...                       ...     ...        ...        ...   
4197  US478160BF06         JOHNSON & JOHNSON  JNJ US 2013-05-12 2016-11-28   
4198  US191216AY64          COCA-COLA CO/THE   KO US 2012-03-14 2018-03-14   
4199  US191216AX81          COCA-COLA CO/THE   KO US 2012-03-14 2015-03-13   
4200  US191216BG40          COCA-COLA CO/THE   KO US 2013-01-11 2020-01-11   
4201  US191216BD19          COCA-COLA CO/THE   KO US 2013-01-11 2016-01-11   

      Tenor FLOATER     Mty Type Currency      BICS Level 1 PRIVATE PLACEMENT  \
0      9.59       N  AT MATURITY      USD            Energy                 N   
1     10.02       N  AT MATURITY      USD       Industrials                 N   
2      9.99       N  AT MATURITY      USD  Consumer Staples                 N   
3     10.06       N  AT MATURITY      USD         Utilities                 N   
4      9.98       N  AT MATURITY      USD    Communications                 N   
...     ...     ...          ...      ...               ...               ...   
4197   2.98       N  AT MATURITY      USD       Health Care                 N   
4198   6.00       N  AT MATURITY      USD  Consumer Staples                 N   
4199   3.00       N  AT MATURITY      USD  Consumer Staples                 N   
4200   7.00       N  AT MATURITY      USD  Consumer Staples                 N   
4201   3.00       N  AT MATURITY      USD  Consumer Staples                 N   

     MAKE WHOLE CALL CALLED DEFAULT DATE  ANNO  MESE COUNTRY RATING S&P  \
0                NaN      N          NaN  2012     2      US       CCC+   
1                NaN      N          NaN  2011     9      US         A-   
2                NaN      N          NaN  2011     8      US         A+   
3                NaN      N          NaN  2011     8      US       BBB-   
4                NaN      N          NaN  2011     8      US       BBB+   
...              ...    ...          ...   ...   ...     ...        ...   
4197             NaN      N          NaN  2013    12      US        AAA   
4198             NaN      N          NaN  2012     3      US         A+   
4199             NaN      N          NaN  2012     3      US         A+   
4200             NaN      N          NaN  2013    11      US         A+   
4201             NaN      N          NaN  2013    11      US         A+   

     RATING MOODY  
0             NaN  
1              WR  
2             NaN  
3            Baa3  
4             NaN  
...           ...  
4197          Aaa  
4198          NaN  
4199          NaN  
4200          NaN  
4201          NaN  

[4202 rows x 19 columns]

In [9]:
con = pdblp.BCon(debug=False, port=8194, timeout=10000)
con.start()

In [29]:
start=1258
end=len(data)
last_date = pd.to_datetime("20210831")

flds = [
    "Z_SPRD_MID",
    "PX_BID",
    "PX_ASK"
]

equity_flds=[
    "PX_LAST",
    "VOLATILITY_30D",
    "CUR_MKT_CAP",
    "TOT_COMMON_EQY",
    "SHORT_AND_LONG_TERM_DEBT"
]
   
    
elms = [
    ('periodicitySelection', 'MONTHLY'),
    ("nonTradingDayFillOption","ALL_CALENDAR_DAYS"),
    ("nonTradingDayFillMethod","PREVIOUS_VALUE")
]
ovrds = [('PRICING_SOURCE', 'BGN')]

In [30]:
all_data={}
for index, row in data[start:end].iterrows():
    # ticker = f"{row['ISIN']}@{pcs} Corp"
    ticker = f"{row['ISIN']} Corp"
    start_date = f"{row['ISSUE DATE']:%Y%m%d}"
    end_date = f"{min(row['MATURITY'], last_date):%Y%m%d}"
    
    bond_data=con.bdh(
        tickers=ticker, 
        flds=flds, 
        start_date=start_date, 
        end_date=end_date, 
        elms=elms,
        ovrds=ovrds
    )

    equity_ticker=f"{row['Ticker']} Equity"
  
    equity_data=con.bdh(
        tickers=equity_ticker, 
        flds=equity_flds, 
        start_date=start_date, 
        end_date=end_date, 
        elms=elms
    )   
    all_data[row["ISIN"]]=[bond_data, equity_data]
    print(row["ISIN"])
    if bond_data.empty:
        print("empty bond data")

USU75130AA97
empty bond data
US14912HJQ48
empty bond data
USU3906HAA69
empty bond data
US68389XBK00
US14913Q2N89
US717081EM19
US24422EUK36
US14912L6U06
US037833AR12
USG7709QAC80
US037833AQ39
US723787AL19
USU1109MAF33
US075887BE86
US191216BV17
USU16353AB74
empty bond data
US14912L6E63
US17275RAR30
US345397XK41
US98978VAQ68
US48203RAL87
USU04167AC34
empty bond data
USG7709QAA25
US55616XAN75
US247361ZM39
USU02320AD80
US69371RN448
USU3002LAA09
US595017AN45
US855244AL32
US36962G7G36
US126650CT50
USU02320AF39
USU0385PAL05
empty bond data
US17275RAU68
USU5254FAC96
US855244AJ85
USU02320AG12
USU2201LAB90
empty bond data
US68389XBA28
US60871RAF73
US88579YAP60
USU0740WAF87
empty bond data
US369550BE71
US512807AR99
USU0018LAD20
US31562QAC15
US17275RBD35
US037833BQ20
US294429AM71
US166764BG48
US11134LAK52
US345397YE71
US37045XCJ37
USU6006PAG73
empty bond data
USG79456AJ12
US02079KAA51
USU59332AD52
empty bond data
US17275RAP73
USU0379LAA18
empty bond data
USU5254FAA31
USU5254FAB14
US25468PDJ21
US571

In [31]:
bond_equity_data =[pd.concat([all_data[data.loc[i, "ISIN"]][0], all_data[data.loc[i, "ISIN"]][1]], axis = 1).droplevel(0,1) for i in range(1261,len(data))]

In [32]:
bond_equity_data=pd.concat(bond_equity_data,keys=data.loc[1261:,"ISIN"])

In [33]:
bond_equity_data.to_csv("bond_equity_data2.csv")

In [ ]:
pd.concat(all_data[data.loc[4377, "ISIN"]], axis=1)[-50:] #.to_csv("test.csv")

In [ ]:
common_tickers=[
    "USGG10YR INDEX",
    "USGG2YR INDEX",
    "SPX INDEX",
    "NAPMPMI Index",
    "VIX Index",
    "USSOC CURNCY",
    "US0003M Index",
    "BSPGCPUS Index"   
]
common_data=con.bdh(
    tickers=common_tickers, 
    flds="PX_LAST", 
    start_date="20070101", 
    end_date="20210831", 
    elms=elms,
)

common_vol=con.bdh(
    tickers="SPX INDEX", 
    flds="VOLATILITY_30D", 
    start_date="20070101", 
    end_date="20210831", 
    elms=elms,
)

In [ ]:
pd.concat([common_data, common_vol], axis=1).to_csv("common_data.csv", )

In [4]:
data = pd.read_csv('new_data.csv', parse_dates=True, index_col=[0, 1])

In [5]:
data

PX_BID   PX_ASK  Z_SPRD_MID  PX_LAST  \
             date                                                
US00101JAH95 2018-02-28   97.572   98.106     182.897    10.56   
             2018-03-31   94.004   94.490     270.527     7.93   
             2018-04-30   93.669   94.164     258.772     8.91   
             2018-05-31   91.285   91.745     327.467     7.52   
             2018-06-30   93.613   94.080     267.007     8.65   
...                          ...      ...         ...      ...   
XS1678630432 2021-04-30  102.220  102.352      21.913   186.02   
             2021-05-31  102.255  102.347      17.411   178.65   
             2021-06-30  101.982  102.097      20.624   175.77   
             2021-07-31  101.901  102.008      20.811   176.02   
             2021-08-31  101.804  101.902      17.120   181.30   

                         VOLATILITY_30D  CUR_MKT_CAP  TOT_COMMON_EQY  \
             date                                                      
US00101JAH95 2018-02-28          60.217    7908.9054        3433.112   
             2018-03-31          63.525    5940.6140        4723.636   
             2018-04-30          56.023    6674.7630        4723.636   
             2018-05-31          50.891    5643.3934        4723.636   
             2018-06-30          41.654    6491.4033        4661.842   
...                                 ...          ...             ...   
XS1678630432 2021-04-30          17.133  337675.3905       85540.000   
             2021-05-31          18.781  324594.9122       85540.000   
             2021-06-30          13.810  319362.1478       85540.000   
             2021-07-31          24.830  319816.3808       86741.000   
             2021-08-31          16.663  329445.0517       86741.000   

                         SHORT_AND_LONG_TERM_DEBT  USGG10YR INDEX|PX_LAST  \
             date                                                           
US00101JAH95 2018-02-28                 10851.635                  2.8606   
             2018-03-31                 10284.582                  2.7389   
             2018-04-30                 10284.582                  2.9531   
             2018-05-31                 10284.582                  2.8586   
             2018-06-30                 10306.437                  2.8601   
...                                           ...                     ...   
XS1678630432 2021-04-30                 56146.000                  1.6259   
             2021-05-31                 56146.000                  1.5943   
             2021-06-30                 56146.000                  1.4680   
             2021-07-31                 55838.000                  1.2223   
             2021-08-31                 55838.000                  1.3088   

                         USGG2YR INDEX|PX_LAST  SPX INDEX|PX_LAST  \
             date                                                   
US00101JAH95 2018-02-28                 2.2500            2713.83   
             2018-03-31                 2.2661            2640.87   
             2018-04-30                 2.4879            2648.05   
             2018-05-31                 2.4274            2705.27   
             2018-06-30                 2.5282            2718.37   
...                                        ...                ...   
XS1678630432 2021-04-30                 0.1584            4181.17   
             2021-05-31                 0.1407            4204.11   
             2021-06-30                 0.2486            4297.50   
             2021-07-31                 0.1839            4395.26   
             2021-08-31                 0.2093            4522.68   

                         NAPMPMI Index|PX_LAST  VIX Index|PX_LAST  \
             date                                                   
US00101JAH95 2018-02-28                   60.8              19.85   
             2018-03-31                   59.2              19.97   
             2018-04-30                   58.6        

In [33]:
n_bonds = len(data.index.get_level_values(0).unique())

In [34]:
fld_sp="RTG_SP_LT_LC_ISSUER_CREDIT"
fld_mdy="RTG_MDY_ISSUER"
ovrd="RATING_AS_OF_DATE_OVERRIDE"

subset = {}
for i, (index, group) in enumerate(data.groupby(level=0)):
    print(f"{i} / {n_bonds}: {index}")
    group = group.droplevel(0)
    ticker = f'{index} Corp'
    ratings = {}
    for date in group.index:
        date_str = f'{date:%Y%m%d}'
        rating = con.ref(ticker, fld_sp, [(ovrd, date_str)])
        if pd.isna(rating.loc[0, 'value']) or (rating.loc[0, 'value'] == "NR"):
            rating = con.ref(ticker, fld_mdy, [(ovrd, date_str)])
        
        ratings[date] = rating.loc[0, 'value']
        
    ratings = pd.Series(ratings, name="RATING")
    subset[index] = pd.concat([group, ratings], axis=1)
  


0 / 2984: US00101JAH95
1 / 2984: US001192AG81
2 / 2984: US001546AU45
3 / 2984: US00164VAD55
4 / 2984: US00164VAE39
5 / 2984: US00165AAH14
6 / 2984: US00165CAB00
7 / 2984: US00165CAD65
8 / 2984: US00206RAM43
9 / 2984: US00206RAV42
10 / 2984: US00206RAY80
11 / 2984: US00206RBF82
12 / 2984: US00206RBM34
13 / 2984: US00206RBN17
14 / 2984: US00206RCA86
15 / 2984: US00206RCC43
16 / 2984: US00206RCE09
17 / 2984: US00206RCM25
18 / 2984: US00206RCN08
19 / 2984: US00206RCT77
20 / 2984: US00206RDC34
21 / 2984: US00206RDD17
22 / 2984: US00206RDP47
23 / 2984: US00206RDQ20
24 / 2984: US00206RGF38
25 / 2984: US00206RGG11
26 / 2984: US00206RGH93
27 / 2984: US00206RGJ59
28 / 2984: US00206RGK23
29 / 2984: US00206RGL06
30 / 2984: US00206RGM88
31 / 2984: US00206RGN61
32 / 2984: US00206RHJ41
33 / 2984: US00206RHN52
34 / 2984: US00206RHP01
35 / 2984: US00206RHQ83
36 / 2984: US00206RHR66
37 / 2984: US00206RHS40
38 / 2984: US00206RHT23
39 / 2984: US00206RHV78
40 / 2984: US00206RHW51
41 / 2984: US00206RHX35
42

In [35]:
new_data = pd.concat(subset)
new_data.to_csv('new_data.csv')